<a href="https://colab.research.google.com/github/Uriel1201/HelloSQL2.0/blob/main/12_hacker_scores/query.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#12. Hacker Scores.
**Writing a query to return the hacker ID, name, and
total score (the sum of maximum scores
for each challenge completed) ordered by descending score,
and by ascending hacker ID in the case of score tie.
Do not display entries for hackers with a score
of zero.**

In [1]:
!pip install polars
import pandas as pd
import numpy as  np
import polars as pl

In [2]:
data1 = {'HACKER_ID' : [1,2,3,4],
         'NAME'      : ['John','Jane','Joe','Jim']
        }

data2 = {'SUB_ID'       : [101,102,103,104,105,106,107,108,109],
         'HACKER_ID'    : [1,1,2,2,2,3,3,3,4],
         'CHALLENGE_ID' : [1,1,1,1,2,1,2,2,1],
         'SCORE'        : [10,12,11,9,13,9,12,15,0]
        }

pd_hackers=pd.DataFrame(data1)
pd_submissions=pd.DataFrame(data2)
pl_hackers=pl.LazyFrame(data1)
pl_submissions=pl.LazyFrame(data2)

In [3]:
print(f'----PANDAS----\nhackers table:\n{pd_hackers}')

----PANDAS----
hackers table:
   HACKER_ID  NAME
0          1  John
1          2  Jane
2          3   Joe
3          4   Jim


In [4]:
print(f'submissions table:\n{pd_submissions}')

submissions table:
   SUB_ID  HACKER_ID  CHALLENGE_ID  SCORE
0     101          1             1     10
1     102          1             1     12
2     103          2             1     11
3     104          2             1      9
4     105          2             2     13
5     106          3             1      9
6     107          3             2     12
7     108          3             2     15
8     109          4             1      0


In [5]:
pd_df1=(pd_submissions[['HACKER_ID','CHALLENGE_ID','SCORE']].groupby(['HACKER_ID','CHALLENGE_ID'],
                                                                     as_index=False
                                                             )
                                                            .agg(MAX_SCORE=('SCORE',
                                                                            'max')
                                                             )
                                                            .groupby('HACKER_ID',
                                                                     as_index=False
                                                             )
                                                            .agg(TOTAL=('MAX_SCORE',
                                                                        'sum')
                                                             )
)
print(f'Identifying max score achieved by each user:\n{pd_df1}')

Identifying max score achieved by each user:
   HACKER_ID  TOTAL
0          1     12
1          2     24
2          3     24
3          4      0


In [6]:
pd_scores=(pd.merge(pd_hackers,
                    pd_df1,
                    on='HACKER_ID',
                    how='left'
              )
             .query('TOTAL>0')
             .sort_values(by=['TOTAL','HACKER_ID'],
                          ascending=[False,True]
              )
)
print(f"Querying users who don't have scores equal to zero:\n{pd_scores}")

Querying users who don't have scores equal to zero:
   HACKER_ID  NAME  TOTAL
1          2  Jane     24
2          3   Joe     24
0          1  John     12


In [7]:
sample1=pl_hackers.head(5)
print(f'----POLARS----\nHackers table (SAMPLE):\n{sample1.collect()}')

----POLARS----
Hackers table (SAMPLE):
shape: (4, 2)
┌───────────┬──────┐
│ HACKER_ID ┆ NAME │
│ ---       ┆ ---  │
│ i64       ┆ str  │
╞═══════════╪══════╡
│ 1         ┆ John │
│ 2         ┆ Jane │
│ 3         ┆ Joe  │
│ 4         ┆ Jim  │
└───────────┴──────┘


In [8]:
sample2=pl_submissions.head(5)
print(f'Submissions table (SAMPLE):\n{sample2.collect()}')

Submissions table (SAMPLE):
shape: (5, 4)
┌────────┬───────────┬──────────────┬───────┐
│ SUB_ID ┆ HACKER_ID ┆ CHALLENGE_ID ┆ SCORE │
│ ---    ┆ ---       ┆ ---          ┆ ---   │
│ i64    ┆ i64       ┆ i64          ┆ i64   │
╞════════╪═══════════╪══════════════╪═══════╡
│ 101    ┆ 1         ┆ 1            ┆ 10    │
│ 102    ┆ 1         ┆ 1            ┆ 12    │
│ 103    ┆ 2         ┆ 1            ┆ 11    │
│ 104    ┆ 2         ┆ 1            ┆ 9     │
│ 105    ┆ 2         ┆ 2            ┆ 13    │
└────────┴───────────┴──────────────┴───────┘


In [19]:
df1=(sample2.select(pl.col('HACKER_ID'),
                    pl.col('CHALLENGE_ID'),
                    pl.col('SCORE')
             )
            .group_by(['HACKER_ID','CHALLENGE_ID'])
            .agg(MAX_SCORE=pl.max('SCORE')
             )
            .group_by('HACKER_ID')
            .agg(TOTAL_SCORE=pl.sum('MAX_SCORE'))
).collect()
print(f'identifying total score by each user:\n{pl_df1}')

identifying total score by each user:
shape: (2, 2)
┌───────────┬─────────────┐
│ HACKER_ID ┆ TOTAL_SCORE │
│ ---       ┆ ---         │
│ i64       ┆ i64         │
╞═══════════╪═════════════╡
│ 1         ┆ 12          │
│ 2         ┆ 24          │
└───────────┴─────────────┘


In [28]:
scores=(pl_submissions.select(pl.col('HACKER_ID'),
                       pl.col('CHALLENGE_ID'),
                       pl.col('SCORE')
                )
               .group_by(['HACKER_ID','CHALLENGE_ID'])
               .agg(MAX_SCORE=pl.max('SCORE')
                )
               .group_by('HACKER_ID')
               .agg(TOTAL_SCORE=pl.sum('MAX_SCORE')
                )
               .join(pl_hackers,
                     on='HACKER_ID',
                     how='right'
                )
               .filter(pl.col('TOTAL_SCORE')>0)
               .sort(by=['TOTAL_SCORE','HACKER_ID'],
                     descending=[True,False]
                )
).collect()
print(f"Querying users who don't have score equal to zero:\n{scores}")

Querying users who don't have score equal to zero:
shape: (3, 3)
┌─────────────┬───────────┬──────┐
│ TOTAL_SCORE ┆ HACKER_ID ┆ NAME │
│ ---         ┆ ---       ┆ ---  │
│ i64         ┆ i64       ┆ str  │
╞═════════════╪═══════════╪══════╡
│ 24          ┆ 2         ┆ Jane │
│ 24          ┆ 3         ┆ Joe  │
│ 12          ┆ 1         ┆ John │
└─────────────┴───────────┴──────┘
